<a href="https://colab.research.google.com/github/bhas10bc/Deep_learning-models/blob/main/Detect_AI_Generated_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load the dataset
train_essays = pd.read_csv('train_essays.csv')
test_essays = pd.read_csv('test_essays.csv')
train_prompts = pd.read_csv('train_prompts.csv')

# Merge essay data with corresponding prompts
train_data = train_essays.merge(train_prompts, on='prompt_id', how='left')

# Preprocess the text data
max_length = 100  # Maximum sequence length
vocab_size = 10000  # Vocabulary size
embedding_dim = 128  # Embedding dimension

# Tokenize the text
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['text'])

# Convert text to sequences
X = tokenizer.texts_to_sequences(train_data['text'])
X_test = tokenizer.texts_to_sequences(test_essays['text'])

# Pad sequences to a fixed length
X = pad_sequences(X, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

# Labels
y = train_data['generated']

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(keras.layers.LSTM(64))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

# Make predictions on the test data
predictions = model.predict(X_test)

# Prepare submission file
submission = pd.DataFrame({'id': test_essays['id'], 'generated': predictions.reshape(-1)})
submission.to_csv('submission.csv', index=False)


Epoch 1/10
35/35 [==============================] - 7s 114ms/step - loss: 0.2449 - accuracy: 0.9837 - val_loss: 0.0297 - val_accuracy: 0.9964
Epoch 2/10
35/35 [==============================] - 4s 106ms/step - loss: 0.0155 - accuracy: 0.9982 - val_loss: 0.0269 - val_accuracy: 0.9964
Epoch 3/10
35/35 [==============================] - 4s 121ms/step - loss: 0.0151 - accuracy: 0.9982 - val_loss: 0.0253 - val_accuracy: 0.9964
Epoch 4/10
35/35 [==============================] - 3s 92ms/step - loss: 0.0139 - accuracy: 0.9982 - val_loss: 0.0254 - val_accuracy: 0.9964
Epoch 5/10
35/35 [==============================] - 3s 88ms/step - loss: 0.0140 - accuracy: 0.9982 - val_loss: 0.0245 - val_accuracy: 0.9964
Epoch 6/10
35/35 [==============================] - 4s 117ms/step - loss: 0.0133 - accuracy: 0.9982 - val_loss: 0.0248 - val_accuracy: 0.9964
Epoch 7/10
35/35 [==============================] - 4s 105ms/step - loss: 0.0132 - accuracy: 0.9982 - val_loss: 0.0252 - val_accuracy: 0.9964
Epoch 8/